In [1]:
!pip install xml
!pip install re
!pip install datetime
!pip install requests

ERROR: Could not find a version that satisfies the requirement xml (from versions: none)
ERROR: No matching distribution found for xml


^C


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


     ---------------------------------------- 0.0/45.6 kB ? eta -:--:--
     ---------------------------------- --- 41.0/45.6 kB 960.0 kB/s eta 0:00:01
     -------------------------------------- 45.6/45.6 kB 751.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/52.6 kB ? eta -:--:--
   -------------------------------------- - 51.2/52.6 kB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 52.6/52.6 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/212.5 kB ? eta -:--:--
   ------- ------------------------------- 41.0/212.5 kB 991.0 kB/s eta 0:00:01
   ----------------- ---------------------- 92.2/212.5 kB 1.1 MB/s eta 0:00:01
   --------------------- ---------------- 122.9/212.5 kB 798.9 kB/s eta 0:00:01
   ------------------------------- ------ 174.1/212.5 kB 952.6 kB/s eta 0:00:01
   -------------------------------------- 212.5/212.5 kB 996.2 kB/s eta 0:00:00


In [ ]:
import xml.etree.ElementTree as ET
import re
from datetime import datetime
import requests

# URL에서 XML 데이터 가져오기
url = "https://www.esgtoday.com/feed/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    xml_data = response.text
else:
    raise Exception(f"Failed to fetch data from {url}. Status code: {response.status_code}")

# XML 데이터를 파싱
root = ET.fromstring(xml_data)

# HTML 태그 제거 함수
def remove_html_tags(text):
    clean = re.sub(r'<.*?>', '', text)
    return clean

# 문장 분리 함수
def split_into_sentences(text):
    # 구두점을 기준으로 문장 분리
    sentences = re.split(r'(?<=[.!?])\s+', text)
    # 불완전한 텍스트도 포함해 최소한의 출력 보장
    return [s.strip() for s in sentences if s.strip()]

# 추출한 데이터를 저장할 리스트
articles = []

# XML 데이터에서 필요한 정보 추출
for item in root.findall(".//item"):
    title = item.find("title").text if item.find("title") is not None else ""
    link = item.find("link").text if item.find("link") is not None else ""
    description = item.find("description").text if item.find("description") is not None else ""
    content_encoded = item.find("{http://purl.org/rss/1.0/modules/content/}encoded")
    content = content_encoded.text if content_encoded is not None else ""
    pub_date = item.find("pubDate").text if item.find("pubDate") is not None else ""

    # HTML 태그 제거 및 상위 5문장만 추출
    combined_content = remove_html_tags(description + " " + content)
    if combined_content:
        sentences = split_into_sentences(combined_content)
        summary = ' '.join(sentences[:5])  # 최대 5개의 문장 포함
    else:
        summary = "No description available."

    # 날짜 변환
    try:
        date_obj = datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %z')
        formatted_date = date_obj.strftime('%Y-%m-%d')
    except ValueError:
        formatted_date = ""

    # JSON 형식으로 저장
    articles.append({
        "title": title,
        "link": link,
        "summary": summary,
        "date": formatted_date,
        "media_name": "ESG Today"
    })

# 결과 출력
import json
print(json.dumps(articles, indent=4, ensure_ascii=False))


[
    {
        "title": "Wells Fargo Leaves Climate-Focused Banking Initiative",
        "link": "https://www.esgtoday.com/wells-fargo-leaves-climate-focused-banking-initiative/?utm_source=rss&utm_medium=rss&utm_campaign=wells-fargo-leaves-climate-focused-banking-initiative",
        "summary": "Wells Fargo has decided to exit the Net-Zero Banking Alliance (NZBA), marking the second major [&#8230;] Wells Fargo has decided to exit the Net-Zero Banking Alliance (NZBA), marking the second major departure from the UN-backed coalition of banks dedicated to advancing global net zero goals through their financing activities, following Goldman Sachs earlier this month. Founded in 2021, the Net-Zero Banking Alliance is a coalition of more than 140 banks spanning 44 countries and representing approximately $73 trillion in assets. Members of the NZBA commit to transitioning operational and attributable greenhouse gas (GHG) emissions from their financing activities to align with net zero pathways